In [1]:
from datetime import datetime, timedelta
import warnings
import numpy as np
warnings.filterwarnings('ignore')
import pandas as pd
import xarray as xr
import os

#import configuration location and filepath
from myconfig import *
output_path = output_path_goes_data

In [2]:

#testing
date_today = now = datetime.now() #datetime(2023, 12, 12) 

# Get today's date
#date_today = datetime.today()
lyr,imon,idym = str(date_today.year), str(date_today.month), str(date_today.day)

init = 0
for incr in range(-6,1):
    date_proc = date_today + timedelta(days=incr)
    date_str1 = date_proc.strftime("%Y%m%d")
    date_proc = date_today + timedelta(days=incr+1)
    date_str2 = date_proc.strftime("%Y%m%d")
    nc_fname = output_path_goes_data+'goes_timeseries_'+target_name+'_time_'+date_str1+'-'+date_str2+'.nc'
    if not(os.path.exists(nc_fname)):
        print('NOTreading',nc_fname)
        continue
    print('reading',nc_fname)
    with xr.open_dataset(nc_fname) as ds:
        ds = ds.drop_vars({'y_image','x_image','y','x'})#.rename({'t':'time'})
        if init==1:
            all_data = xr.concat([all_data,ds], dim="time")
        else:
            all_data = ds
            init = 1

reading ./../data/goes/data/goes_timeseries_MilwaukeeTrench_time_20241020-20241021.nc


reading ./../data/goes/data/goes_timeseries_MilwaukeeTrench_time_20241021-20241022.nc
reading ./../data/goes/data/goes_timeseries_MilwaukeeTrench_time_20241022-20241023.nc
reading ./../data/goes/data/goes_timeseries_MilwaukeeTrench_time_20241023-20241024.nc
reading ./../data/goes/data/goes_timeseries_MilwaukeeTrench_time_20241024-20241025.nc
reading ./../data/goes/data/goes_timeseries_MilwaukeeTrench_time_20241025-20241026.nc
NOTreading ./../data/goes/data/goes_timeseries_MilwaukeeTrench_time_20241026-20241027.nc


In [3]:
all_data['time']=all_data.t
subset = all_data.sel(step=18)
ds_norm = subset.CMI_C10-subset.CMI_C10.min()
ds_norm = ds_norm/ds_norm.max()

ds = ds_norm

start_time = pd.to_datetime(ds.t[0].data).strftime('%Y-%m-%dT%H:%M:%S')
end_time = pd.to_datetime(ds.t[-1].data).strftime('%Y-%m-%dT%H:%M:%S')
new_time = pd.date_range(start=start_time, end=end_time, periods=7200).to_numpy()
ds_interp = ds.interp(time=new_time)

ds = ds_interp
ds[0,:]=ds[1,:]
fname = output_path_goes_data+'/goes_timeseries_nrt.nc'
ds.to_netcdf(fname)
df = pd.DataFrame({'time':ds.time.data,
                   'N':ds[:,0].data,'NE':ds[:,1].data,
                   'E':ds[:,2].data,'SE':ds[:,3].data,
                   'S':ds[:,4].data,'SW':ds[:,5].data,
                   'W':ds[:,6].data,'NW':ds[:,7].data})
fname = output_path_nrt+'/goes_timeseries_nrt.csv'
df.to_csv(fname,index=False)
print('done')

done


In [4]:
#subset = all_data.sel(step=12)
#norm_subset = subset.CMI_C10-subset.CMI_C10.min()
#norm_subset = norm_subset/norm_subset.max()
#norm_subset.plot()